## get Cause efffect phrase from the sentence.

In [1]:
#filtered_rows.reset_index(drop=True, inplace=True)


In [2]:
#load CE semeval dataset
import pandas as pd
#sem_CE_test= pd.read_csv('data/in/semeval/sem_CE_test.csv')
sem_CE_test= pd.read_csv('data/in/KNN_RAG/ADE_CE_extract_TEST.csv')

pm_DB = sem_CE_test

pm_DB = pm_DB.rename(columns={'cause': 'cause_t', 'effect': 'effect_t'})
pm_DB

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,..."
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ..."
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ..."
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ..."
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,..."
...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934..."
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064..."
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,..."
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ..."


In [3]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [4]:
import groq
from groq import Groq

client = groq.Groq(api_key= "enter your API key here..")#my iu mail key


#gemma2-9b-it# llama3-8b-8192
#"" 
#mixtral-8x7b-32768

In [5]:
def C_E_CP_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause": "", "effect": "", "causality_phrase": ""}.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
                   
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
                   
          ''' + fewshot_examples
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="gemma2-9b-it",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [6]:
def get_fewshotexamples_C_E_CP(fewshot_index):

    
    #fewshot_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    
    train_sentence = semeval_data['sentence'].to_list()
    train_causalityPhrase = semeval_data['causality_phrase'].to_list()

    fewshot_example_sentence = [train_sentence[i] for i in fewshot_index]
    fewshot_example_causalityPhrase = [train_causalityPhrase[i] for i in fewshot_index]


    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_example_causalityPhrase[i]} 
                    for i, sentence in enumerate(fewshot_example_sentence)}
    
    # Assuming json_data is already defined
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples

In [7]:
# semeval
filtered_rows = pm_DB

In [8]:
#load semeval sentence and patterns.
semeval_data = pd.read_csv('data/in/RAG/comb_SCITEsemADE_CausalityPattern.csv')# 

semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

In [9]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        #print('ind :', ind)
        window_size = len(p)

        # Slide through input_sentence with a window size 
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('causality phrase: ',p, 'index is', ind)
                #print(f"Window '{window}' - LS: {score}")

    #print('fewshot_index', fewshot_index)
    fewshot_index = list(set(fewshot_index))
    #print('fewshot_index', fewshot_index)
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [10]:
import json

cause_list = []
effect_list = []
causality_phrase_list = []
fewshotfound_inputsentence_index=[]
for i in range (len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    # few shot examples augmentation.
    fewshot_index=[]
    fewshot_index = get_fewshot_examples_index(input_sentence)#########FEWSHOT
    
    ##
    if len(fewshot_index) == 0:
        fewshot_index = [k for k in range(10)]
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
    else:
        fewshotfound_inputsentence_index.append(i)
        
        #fewshot_index = [k for k in range(10)]###############################RANDOMSHOT-ZERO
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
        
    ##
    print('fewshot_index :', fewshot_index)
    ##############print('fewshot_examples', fewshot_examples)

    #fewshot_examples = get_fewshotexamples_C_E_CP(fewshot_index) 
    
    #print('fewshotfound_inputsentence_index', fewshotfound_inputsentence_index)
    
    # output generation
    try:
        out = C_E_CP_Identifier(input_sentence, fewshot_examples)

        print('out: ', out)

        json_out = json.loads(out)
        cause = json_out['cause']
        effect = json_out['effect']
        causality_phrases = json_out['causality_phrase']
        
    except TypeError as e:
        print('TypeError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
        
    except KeyError as e:
        print('KeyError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
    
    except groq.APIConnectionError as e:
        print('APIConnectionError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
        
    except groq.RateLimitError as e:
        print('RateLimitError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')       
        continue
        
    except groq.APIStatusError as e:
        print('APIStatusError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
    

    
    cause_list.append(cause)
    effect_list.append(effect)
    causality_phrase_list.append(causality_phrases)
    
    #print('input_sentence: ', input_sentence )
    #print('fewshot_examples: ', fewshot_examples )
    #print('cause: ', cause)
    #print('effect: ', effect)
    #print('causality_phrase: ', causality_phrases)
    
    
    print(i)
    #if i==0:
        #break;
    #print(causality_identifier)
    
    

fewshot_index : [2341, 2337, 2340, 2142, 2141, 1551, 2135, 2140, 2139, 1241]
out:  {"cause": "5-fluorouracil", "effect": "Squamous-cell carcinoma arising in a basal-cell epithelioma", "causality_phrase": "treated with"}
0
fewshot_index : [2091, 1551, 2088, 1533, 280, 1511, 2092, 2119, 1139, 2086]
out:  {"cause": "long-term therapy with mitomycin C and 5-fluorouracil", "effect": "acute renal toxicity", "causality_phrase": "in patients receiving"}
1
fewshot_index : [1533, 2086, 518, 1136, 1551, 522, 1513, 523, 2089, 626]
out:  {"cause": "budesonide", "effect": "contact dermatitis", "causality_phrase": "causing"}
2
fewshot_index : [12, 1551, 2088, 15, 1511, 1506, 1510, 13, 2085, 985]
out:  {"cause": "mannitol-induced ARF", "effect": "rapid reversal", "causality_phrase": "for"}
3
fewshot_index : [1531, 765, 2209, 769, 2090, 2091, 1242, 2087, 1533, 763]
out:  {"cause": "discontinued the chlorambucil", "effect": "clinical AML", "causality_phrase": "After"}
4
fewshot_index : [765, 2333, 770, 

fewshot_index : [1507, 279, 2086, 1531, 1532, 2088, 829, 1506, 821, 1530]
out:  {"cause": "lithium therapy", "effect": "renal failure", "causality_phrase": "during"}
41
fewshot_index : [2088, 765, 2334, 2337, 827, 2094, 775, 830, 2086, 823]
out:  {"cause": "methotrexate therapy", "effect": "accelerated nodulosis", "causality_phrase": "during"}
42
fewshot_index : [1515, 2333, 384, 2334, 393, 2339, 103, 1512, 389, 0]
out:  {"cause": "administration of amantadine", "effect": "psychotic decompensations", "causality_phrase": "associated with"}
43
fewshot_index : [2336, 2340, 397, 1507, 1509, 2334, 1506, 6, 2338, 387]
out:  {"cause": "the use of intravenous vasopressin", "effect": "Rhabdomyolysis", "causality_phrase": "associated with"}
44
fewshot_index : [767, 1530, 1510, 830, 986, 768, 1529, 1528, 1531, 775]
out:  {"cause": "dexamethasone treatment for bronchopulmonary dysplasia", "effect": "clinically significant septal hypertrophy and left ventricular outflow tract obstruction", "causali

fewshot_index : [2092, 1531, 1506, 2088, 1871, 1509, 1869, 2087, 1865, 2085]
out:  {"cause": "acute vitamin D intoxication", "effect": "hypercalcemia", "causality_phrase": "secondary to"}
80
fewshot_index : [1509, 2341, 1527, 2337, 125, 1510, 1551, 2093, 1514, 1518]
out:  {"cause": "tamoxifen therapy", "effect": "Well-differentiated endometrial adenocarcinoma of the secretory type (FIGO Grade 1) with minimal myometrial invasion", "causality_phrase": "on"}
81
fewshot_index : [2092, 814, 1528, 2094, 1529, 816, 815, 1532, 2087, 278]
out:  {"cause": "in utero exposure to phenytoin", "effect": "Basilar invagination and mid-line skeletal abnormalities", "causality_phrase": "due to"}
82
fewshot_index : [1509, 1508, 2093, 1243, 279, 2088, 1506, 2090, 1510, 1507]
out:  {"cause": "", "effect": "", "causality_phrase": ""}
83
fewshot_index : [1775, 2336, 1509, 826, 821, 1514, 1506, 830, 1511, 2335]
out:  {"cause": "low-dose alprazolam treatment", "effect": "Changes in the plasma cortisol level", "

out:  {"cause": "Salicylate", "effect": "hepato toxicity", "causality_phrase": "in"}
119
fewshot_index : [765, 1527, 104, 107, 771, 1530, 1136, 767, 1551, 1514]
out:  {"cause": "chronic administration of methotrexate", "effect": "a shallow erosion of a psoriatic plaque", "causality_phrase": "after chronic administration of"}
120
fewshot_index : [1509, 1164, 127, 118, 212, 1551, 1533, 2338, 2336, 1239]
out:  {"cause": "treatment with halofantrine", "effect": "cardiac effects", "causality_phrase": "after the treatment"}
121
fewshot_index : [2091, 2093, 2092, 1530, 929, 1533, 2087, 2094, 928, 927]
out:  {"cause": "sulindac", "effect": "incapacitating isolated idential sensory neuropathy", "causality_phrase": "exposed to"}
122
fewshot_index : [2092, 2094, 2333, 518, 2340, 2093, 2336, 1241, 2339, 2087]
out:  {"cause": "defective calcium-activated K+ channels", "effect": "rhabdomyolysis", "causality_phrase": "cause"}
123
fewshot_index : [4, 393, 383, 9, 388, 2341, 2335, 394, 385, 2340]
out: 

fewshot_index : [121, 1529, 1532, 127, 1511, 1514, 119, 126, 1533, 1512]
out:  {"cause": "inhalation of normal doses of albuterol", "effect": "hypokalemic", "causality_phrase": "after"}
159
fewshot_index : [1177, 15, 14, 18, 17, 1180, 10, 1172, 19, 1173]
out:  {"cause": "Nitrendipine", "effect": "gingival hyperplasia", "causality_phrase": "induced"}
160
fewshot_index : [2334, 19, 1511, 1746, 14, 1515, 1170, 1531, 1172, 1174]
out:  {"cause": "ciprofloxacin-induced", "effect": "SJS and acute onset of VBDS", "causality_phrase": "induced"}
161
fewshot_index : [985, 180, 1514, 1527, 1510, 122, 2064, 1241, 120, 127]
out:  {"cause": "long term mesalamine therapy for ulcerative colitis", "effect": "acute pancreatitis", "causality_phrase": "after"}
162
fewshot_index : [812, 1506, 2089, 813, 2086, 1511, 1514, 1515, 1532, 1529]
out:  {"cause": "sodium bisulfite in Tathion eye drops", "effect": "contact dermatitis", "causality_phrase": "due to"}
163
fewshot_index : [1527, 6, 2339, 401, 8, 1242, 1,

...

fewshot_index : [1241, 2088, 2092, 1746, 1531, 1136, 1530, 25, 1732, 1729]
out:  {"cause": "various medications, such as Haloperidol", "effect": "NMS", "causality_phrase": "receiving"}
2096
fewshot_index : [985, 2138, 922, 1510, 2341, 1623, 1241, 1532, 1529, 1511]
out:  {"cause": "etanercept for severe plaque psoriasis", "effect": "persistent hypoglycemia requiring the lowering and eventual elimination of his previous insulin regimen", "causality_phrase": "while treated with"}
2097
fewshot_index : [18, 1177, 1514, 1507, 1174, 17, 1509, 1241, 1242, 1508]
out:  {"cause": "ibuprofen", "effect": "meningitis", "causality_phrase": "induced"}
2098
fewshot_index : [1255, 0, 389, 1136, 2091, 2092, 1771, 398, 400, 402]
out:  {"cause": "orally administered terbinafine", "effect": "Hepatobiliary disorders", "causality_phrase": "associated with"}
2099
fewshot_index : [1136, 2340, 2341, 2119, 2333, 1508, 278, 1515, 2335, 2094]
out:  {"cause": "Pharmacokinetic determinants", "effect": "myelotoxicity 

fewshot_index : [1718, 766, 278, 2339, 279, 1242, 280, 1509, 764, 1512]
out:  {"cause": "a large dose of pyridostigmine bromide", "effect": "postoperative psychosis and was diagnosed as having bromide intoxication", "causality_phrase": "received"}
2135
fewshot_index : [1529, 769, 1527, 2333, 762, 2335, 2339, 767, 2334, 1137]
out:  {"cause": "acyclovir overdose", "effect": "coma and nonoliguric renal failure", "causality_phrase": "who developed"}
2136
fewshot_index : [949, 1508, 1511, 1513, 954, 1532, 1510, 2093, 2094, 1533]
out:  {"cause": "corticosteroid therapy", "effect": "Possible recurrence of amiodarone pulmonary toxicity", "causality_phrase": "following"}
2137
fewshot_index : [2086, 1532, 1529, 2089, 1528, 1136, 1509, 1527, 1512, 1533]
out:  {"cause": "lacosamide doses up to 600 mg/day", "effect": "nonspecific central nervous system effects (e.g., dizziness, ataxia, diplopia, and somnolence)", "causality_phrase": "include"}
2138
fewshot_index : [823, 825, 2337, 1530, 827, 1532, 

fewshot_index : [2090, 1170, 1533, 1531, 118, 1241, 122, 2085, 1551, 127]
out:  {"cause": "introduction in 1952", "effect": "symptoms resembling Parkinson's disease", "causality_phrase": "Soon after"}
2174
fewshot_index : [1152, 480, 479, 1137, 278, 1987, 478, 1514, 1511, 1985]
out:  {"cause": "the newer macrolide antibiotics clarithromycin and azithromycin", "effect": "rhabdomyolysis in patients taking lovastatin", "causality_phrase": "precipitated by the use of"}
2175
fewshot_index : [2336, 1509, 1, 392, 3, 2, 0, 387, 1551, 2334]
out:  {"cause": "VRC", "effect": "angio-oedema", "causality_phrase": "associated with"}
2176
fewshot_index : [280, 955, 278, 952, 950, 948, 953, 279, 951, 947]
out:  {"cause": "progestin therapy", "effect": "Extensive squamous metaplasia was found in endometrial glands", "causality_phrase": "following"}
2177
fewshot_index : [123, 1136, 1507, 1518, 126, 1551, 1527, 1509, 119, 121]
out:  {"cause": "40 months on chlorambucil", "effect": "the presence of a balan

out:  {"cause": "Atovaquone", "effect": "vortex keratopathy involving the corneal epithelium", "causality_phrase": "causing"}
2212
fewshot_index : [0, 403, 8, 278, 386, 2, 1241, 9, 383, 388]
out:  {"cause": "cresol", "effect": "MH", "causality_phrase": "could possibly trigger"}
2213
fewshot_index : [2085, 2093, 1136, 2094, 1529, 2340, 1531, 1533, 2337, 1241]
out:  {"cause": "Anastrozole", "effect": "sclerosing glomerulonephritis", "causality_phrase": "may be the causative factor in"}
2214
fewshot_index : [2086, 124, 2087, 127, 280, 119, 123, 121, 118, 279]
out:  {"cause": "low (1 mg/kg) azathioprine dose", "effect": "pancytopenia", "causality_phrase": "after 56 days therapy"}
2215
fewshot_index : [1508, 1512, 13, 278, 2090, 1530, 1529, 1180, 280, 12]
out:  {"cause": "dexamethasone-induced hypertrophic obstructive cardiomyopathy", "effect": "symptomatic beta-blockade", "causality_phrase": "Effect of"}
2216
fewshot_index : [2333, 385, 2337, 2338, 8, 1242, 388, 3, 400, 383]
out:  {"cause"

fewshot_index : [1529, 515, 1422, 1426, 1421, 2087, 1531, 2088, 1241, 279]
out:  {"cause": "Unintended exposure to acyclovir early in pregnancy", "effect": "excessive maternal and physician anxiety", "causality_phrase": "may cause"}
2253
fewshot_index : [551, 1123, 2337, 1241, 1533, 0, 6, 479, 819, 8]
out:  {"cause": "MTX administration", "effect": "Epstein-Barr virus-associated polymorphic lymphoproliferative disorder (LPD)", "causality_phrase": "caused by"}
2254
fewshot_index : [278, 1533, 1510, 1512, 1530, 2087, 2333, 2089, 1551, 1509]
out:  {"cause": "olanzapine", "effect": "rapidly developing hyperglycemia", "causality_phrase": "should be aware of the potential for"}
2255
fewshot_index : [209, 2332, 985, 1529, 280, 2341, 986, 2333, 1990, 2334]
out:  {"cause": "cholestyramine II sachets twice daily for two months", "effect": "lethargy, confusion and drowsiness", "causality_phrase": "After taking"}
2256
fewshot_index : [1175, 1178, 16, 11, 17, 1173, 1172, 10, 1176, 1174]
out:  {"cau

In [11]:
len(filtered_rows)

2271

In [12]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

2270

In [13]:
filtered_rows['cause'] = cause_list
filtered_rows['effect'] = effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",5-fluorouracil,Squamous-cell carcinoma arising in a basal-cel...,treated with
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,in patients receiving
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol-induced ARF,rapid reversal,for
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",discontinued the chlorambucil,clinical AML,After
...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",fludarabine phosphate,respiratory syncytial virus (RSV) infection,treated with
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...",a second course of flurbiprofen,biopsy-proven acute tubulointerstitial nephritis,associated with
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,Onset of male gynaecomastia,treated with


In [14]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [15]:
#get accuracy
from fuzzywuzzy import fuzz

def get_accuracy(filtered_rows):
    prediction= []
    e_prediction=[]
    c_prediction=[]
    for i in range(len(filtered_rows)):
        #print(i)
        cause_pred = filtered_rows['cause'][i]
        cause_t = filtered_rows['cause_t'][i]
        
        effect_pred = filtered_rows['effect'][i]
        effect_t = filtered_rows['effect_t'][i]

        # Slide through input_sentence with a window size of 8
        #print('effect_pred',  effect_pred)
        #print('effect_t',  effect_t)
        cause_pred = str(cause_pred)
        effect_pred=str(effect_pred)

        score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
        #score = fuzz.ratio(cause_pred, cause_t)################medcause
        if score >90:################90 non -medcause
            pred_c = 1
            c_prediction.append(pred_c)
        else:
            pred_c = 0
            c_prediction.append(pred_c)
            
        score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
        #score = fuzz.ratio(effect_pred, effect_t)################medcause

        if score >90: ################90 non -medcause
            pred_e = 1
            e_prediction.append(pred_e)
        else:
            pred_e = 0
            e_prediction.append(pred_e)

        
        if ((pred_e ==1) & (pred_c ==  1)):
            prediction.append(1)
        else:
            prediction.append(0)
            
    

    return prediction, c_prediction, e_prediction


In [16]:
prediction, c_prediction, e_prediction = get_accuracy(filtered_rows)

no pred_phrase
no pred_phrase
prediction length lower renal failure p: nephrotoxicity
prediction length lower chronic renal failure p: acute hemorrhagic gastritis
prediction length lower first-dose phenomenon p: severe drop in blood pressure
prediction length lower hypercalcemia p: vitamin d intoxication
no pred_phrase
no pred_phrase
prediction length lower significant morbidity p: osteonecrosis of the jaw
prediction length lower stroke p: acute retinal ischemic change
prediction length lower aortic wall was damaged p: aortic wall in this case was damaged
prediction length lower serum lithium levels p: increase serum lithium levels
prediction length lower this observation p: electrophysiologic effect
prediction length lower ttp patients with p: thrombotic microangiopathy
prediction length lower myocardial infarction p: arteriosclerotic disease
prediction length lower theophylline intoxication p: decrease in theophylline clearance
no pred_phrase
no pred_phrase
prediction length lower un

In [17]:
sum(prediction)/len(filtered_rows)

0.8260678115367679

In [18]:
sum(c_prediction)/len(filtered_rows)

0.9150154117129018

In [19]:
sum(e_prediction)/len(filtered_rows)

0.8859533245266402

In [20]:
filtered_rows['predictions'] = prediction

In [21]:
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase,predictions
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",5-fluorouracil,Squamous-cell carcinoma arising in a basal-cel...,treated with,1
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,in patients receiving,1
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing,1
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol-induced ARF,rapid reversal,for,0
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",discontinued the chlorambucil,clinical AML,After,1
...,...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",fludarabine phosphate,respiratory syncytial virus (RSV) infection,treated with,1
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to,0
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...",a second course of flurbiprofen,biopsy-proven acute tubulointerstitial nephritis,associated with,1
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,Onset of male gynaecomastia,treated with,1


In [22]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#filtered_rows.to_csv('data/out/RAG_causality/CE_extracts/randomshotCOMB_ADE_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/task2/gemma/rand_ADE_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [23]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
